In [8]:
import torch, os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from dataset import load_data, RubiksDataset
from torch.utils.data import DataLoader
from reward import reward_model_strict
from peft import LoraConfig
import bitsandbytes as bnb

LR = 1.41e-5
BATCH_SIZE = 1
MINI_BATCH_SIZE = 1

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.float16
)

model_id = "google/gemma-2b-it"
#model_id = "vicgalle/gpt2-open-instruct-v1"
config = PPOConfig(
    model_name=model_id,
    learning_rate=LR,
    batch_size=BATCH_SIZE,
    mini_batch_size=MINI_BATCH_SIZE
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/josue/anaconda3/envs/RLHFRubicks/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name, token=os.environ['HF_TOKEN'])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name, quantization_config=bnb_config, peft_config=lora_config, device_map="auto", token=os.environ['HF_TOKEN'])
#model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name,  device_map="auto", token=os.environ['HF_TOKEN'])

optimizer = bnb.optim.Adam8bit(model.parameters(), lr=LR)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
#text = "Write me a poem about Machine Learning. Length of the poem should not exceed 10 words."
with open('prompt.txt', 'r') as file:
    text = file.read()
print(text)

You are a Rubik's cube solving assistant. Your job is to generate the next best move
when solving a Rubik's cube when given the a Rubik's cube scramble. A scramble is a list of
moves that are performed on a fully solved Rubik's cube in order to scramble it up. When replying,
you must only reply with a single move.

Below I describe the possible moves:
U (Up): Rotate the upper face 90 degrees clockwise.
U' (Up Prime): Rotate the upper face 90 degrees counter-clockwise.
U2: Rotate the upper face 180 degrees.
D (Down): Rotate the bottom face 90 degrees clockwise.
D' (Down Prime): Rotate the bottom face 90 degrees counter-clockwise.
D2: Rotate the bottom face 180 degrees.
F (Front): Rotate the front face 90 degrees clockwise.
F' (Front Prime): Rotate the front face 90 degrees counter-clockwise.
F2: Rotate the front face 180 degrees.
B (Back): Rotate the back face 90 degrees clockwise.
B' (Back Prime): Rotate the back face 90 degrees counter-clockwise.
B2: Rotate the back face 180 degrees.


In [4]:
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


You are a Rubik's cube solving assistant. Your job is to generate the next best move
when solving a Rubik's cube when given the a Rubik's cube scramble. A scramble is a list of
moves that are performed on a fully solved Rubik's cube in order to scramble it up. When replying,
you must only reply with a single move.

Below I describe the possible moves:
U (Up): Rotate the upper face 90 degrees clockwise.
U' (Up Prime): Rotate the upper face 90 degrees counter-clockwise.
U2: Rotate the upper face 180 degrees.
D (Down): Rotate the bottom face 90 degrees clockwise.
D' (Down Prime): Rotate the bottom face 90 degrees counter-clockwise.
D2: Rotate the bottom face 180 degrees.
F (Front): Rotate the front face 90 degrees clockwise.
F' (Front Prime): Rotate the front face 90 degrees counter-clockwise.
F2: Rotate the front face 180 degrees.
B (Back): Rotate the back face 90 degrees clockwise.
B' (Back Prime): Rotate the back face 90 degrees counter-clockwise.
B2: Rotate the back face 180 degrees.


In [5]:
data_path = 'datasets/Kociemba_solutions.csv'
data = load_data(data_path)
dataset = RubiksDataset(tokenizer, data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [6]:
ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    tokenizer=tokenizer,
    optimizer= optimizer
)

/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:262: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [7]:
epochs = 1

for epoch in tqdm(range(epochs), "epoch: "):
    for query_tensors, correct_answers in tqdm(dataloader):
        query_tensors = query_tensors.squeeze(1)
        query_tensors = list(torch.unbind(query_tensors, dim=0))
        #### Get response from model
        response_tensors = [ppo_trainer.generate(query_tensor, max_length=570).squeeze(0) for query_tensor in query_tensors]
        responses = [tokenizer.decode(r.squeeze()) for r in response_tensors]

        #### Compute reward score
        rewards = [torch.tensor(reward_model_strict(correct_answer, response), dtype=torch.float16) for correct_answer, response in zip(correct_answers, responses)]

        #### Run PPO step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        #TODO: log stats
        # ppo_trainer.log_stats(stats, batch, rewards)

#### Save model
ppo_trainer.save_model(f"gemma-2b-it-rlhf-kociemba")

  0%|          | 0/1000 [00:00<?, ?it/s]

 U<eos>


/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1285: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  std_scores = data["scores"].std()
/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1312: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1315: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered inter

 U'<eos>


/home/ace/.virtualenvs/RLproject/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -4.88 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(

  0%|          | 2/1000 [00:09<1:17:48,  4.68s/it]

...

 <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>



100%|█████████▉| 999/1000 [4:14:26<00:15, 15.91s/it]

 <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>



epoch: 100%|██████████| 1/1 [4:14:42<00:00, 15282.42s/it]


AttributeError: 'PPOTrainer' object has no attribute 'save_model'

In [8]:
#### Save model
ppo_trainer.save_pretrained(f"gemma-2b-it-rlhf-kociemba")